In [1]:
import click
import keras
import math
import numpy
import tqdm

Using TensorFlow backend.


In [2]:
numpy.set_printoptions(threshold=numpy.nan)

In [3]:
def read_bits(file_path):
    bits = []
    with open(file_path, 'rb') as f:
        bs = (ord(chr(b)) for b in f.read())
        for b in bs:
            for i in reversed(range(8)):
                bits.append((b >> i) & 1)
    return numpy.array(bits)

In [4]:
def next_suitable_square(n):
    return int((math.ceil(n ** (1 / 2) / block_size) * block_size) ** 2)

In [5]:
def next_suitable_cube(n):
    return int((math.ceil(n ** (1 / 3) / block_size) * block_size) ** 3)

In [6]:
def init_model_3d(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    x = keras.layers.Conv3D(8, (input_shape[0] // 2 + 1, input_shape[1] // 2 + 1, input_shape[2] // 2 + 1))(input_layer)
    x = keras.layers.Flatten()(x)
    
    output_layer = keras.layers.Activation('sigmoid')(x)
    
    model = keras.models.Model(input_layer, output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
def random_pad_3d(bits):
    global pad_size
    pad_size = next_suitable_cube(len(bits)) - len(bits)
    pad_array = numpy.random.randint(2, size=(pad_size,))
    return numpy.concatenate([bits, pad_array])[numpy.newaxis, :]

In [8]:
#def cubify(bits):
#    return bits.reshape(blocks, 1, block_size, block_size, block_size, 1)

In [9]:
def generate_key_cube(data): 
    return numpy.random.randint(2, size=(1, block_size, block_size, block_size, 1))

In [10]:
def split_data(data):
    return data.reshape(blocks, 1, data.shape[1] // blocks)

In [11]:
#def training_data_3d(key, data):
#    size = int(numpy.round(len(data) ** (1 / 3)))
#    x = key.reshape(1, size, size, size, 1)
#    y = data.reshape(1, size, size, size, 1)
#    return x, y

In [12]:
def fit_models(x, y):
    models = []
    for i in tqdm.tqdm(range(blocks)):
        model = init_model_3d(x.shape[1:])
        acc = 0
        while acc < 1:
            model.fit(x, y[i, :, :], batch_size=1, epochs=1, verbose=0)
            acc = model.evaluate(x, y[i, :, :], batch_size=1, verbose=0)[1]
            #print("Accuracy:", acc)
        models.append(model)
    return models

In [13]:
block_size = 32

In [14]:
data_bits = read_bits("/home/santiago/Pictures/Wallpapers/test.tar.gz")

In [15]:
data_bits.shape

(5220560,)

In [16]:
data = random_pad_3d(data_bits)

In [17]:
data.shape

(1, 7077888)

In [18]:
blocks = int(numpy.round(data.shape[1] ** (1 / 3) / block_size)) ** 3

In [19]:
key_cube = generate_key_cube(data)

In [20]:
key_cube.shape

(1, 32, 32, 32, 1)

In [21]:
dataset = split_data(data)

In [22]:
dataset.shape

(216, 1, 32768)

In [23]:
#the_model = init_model_3d(key_cubes.shape[2:])

In [24]:
#the_model.summary()

In [25]:
models = fit_models(key_cube, dataset)

  2%|▏         | 4/216 [00:28<25:14,  7.14s/it]

KeyboardInterrupt: 

In [ ]:
#result = numpy.round(the_model.predict(key_cube)).astype(numpy.uint8)

In [ ]:
#numpy.testing.assert_equal(result, data)